# Fifth Task: Explanation

### Import of the needed libraries and the dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import pydotplus 
from IPython.display import Image  
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

cyclists = pd.read_csv('./dataset/cyclists_trasformed.csv')
races = pd.read_csv('./dataset/races_trasformed.csv')

`CHATGPT`:

Per affrontare il prossimo assignment, dovrai focalizzarti sulle spiegazioni dei modelli costruiti in precedenza. Questo include l'analisi delle feature più importanti, delle regole che il modello utilizza per fare previsioni, e dei controfattuali, ossia come cambiare gli input per modificare il risultato del modello. Ecco un piano passo-passo per affrontare l'assegnamento:

---

### 1. **Feature Importance**
- **Obiettivo:** Identificare quali variabili influenzano maggiormente il modello.
- **Cosa fare:**
  - Per modelli ad albero come Random Forest, XGBoost, e Decision Tree, usa attributi come `.feature_importances_` per visualizzare l'importanza delle feature.
  - Per modelli lineari o SVM con kernel lineare, osserva i coefficienti dei pesi delle feature.
  - Per le reti neurali, utilizza tecniche come **SHAP (SHapley Additive exPlanations)** o **LIME (Local Interpretable Model-agnostic Explanations)** per interpretare le feature.
- **Come interpretare:**
  - Verifica se le feature più importanti evidenziate dai modelli corrispondono a quanto scoperto durante la fase di data understanding e clustering. 
  - Cerca conferme di pattern noti, come ad esempio se la qualità del ciclista o il profilo delle gare sono determinanti per la posizione in classifica.

---

### 2. **Rule Explanation**
- **Obiettivo:** Esplicita le regole decisionali del modello.
- **Cosa fare:**
  - Per il modello rule-based (RIPPER), utilizza `.out_model()` per visualizzare le regole.
  - Per modelli come Decision Tree, esporta e visualizza la struttura dell'albero decisionale usando strumenti come `plot_tree` o `graphviz`.
  - Per Random Forest e XGBoost, combina la visualizzazione delle regole con l'importanza delle feature.
- **Come interpretare:**
  - Analizza se le regole create dal modello sono coerenti con i pattern logici e con i risultati della fase di analisi.
  - Cerca regole contro-intuitive o insensate che potrebbero indicare overfitting o bias nei dati.

---

### 3. **Counterfactual Instances**
- **Obiettivo:** Identificare come cambiare gli input per alterare la previsione.
- **Cosa fare:**
  - Genera controfattuali usando librerie come **Alibi** o **DiCE (Diverse Counterfactual Explanations)**.
  - Analizza gli input per cui il modello non prevede un risultato desiderato (es. non in top 20) e valuta quali cambiamenti li porterebbero a raggiungerlo.
- **Come interpretare:**
  - Valuta se i controfattuali sono sensati e realisticamente attuabili. Ad esempio, suggerire che un ciclista cambi la sua altezza per migliorare il ranking non è praticabile.

---

### 4. **Analisi delle Spiegazioni**
Confronta le spiegazioni in base a:
- **Proprietà proprie:**
  - **Fedeltà (fidelity):** Le spiegazioni rispecchiano accuratamente il comportamento del modello?
  - **Complessità (complexity):** Le spiegazioni sono comprensibili o troppo complesse? Le regole hanno una lunghezza accettabile? Le feature più importanti sono intuitive?
- **Relazione con Data Understanding e Clustering:**
  - Confronta le spiegazioni con le osservazioni derivanti dall'analisi esplorativa dei dati e dal clustering. 
  - Verifica:
    - Se il modello ha imparato gli stessi pattern evidenziati nell’analisi dei dati (es. nazionalità, peso/altezza).
    - Se ci sono nuovi pattern non considerati precedentemente.
    - Se ci sono pattern insensati o non attesi.

---

### 5. **Documentazione**
- **Report delle Spiegazioni:** Scrivi una documentazione chiara che includa:
  - Visualizzazioni (grafici di importanza delle feature, regole estratte, controfattuali).
  - Confronti qualitativi e quantitativi tra i modelli.
  - Commenti su pattern emersi e loro coerenza con l'analisi precedente.
  - Considerazioni su eventuali limiti dei modelli o problematiche nei dati.

---


4. Alternative: Usare %run
Se preferisci mantenere il file originale come notebook (fourth_task.ipynb) senza convertirlo in .py, puoi eseguire direttamente il file Jupyter con il comando %run.

Esempio:

Nel file fifth_task.ipynb, esegui:

# Esegui il notebook fourth_task.ipynb
%run fourth_task.ipynb

# Ora puoi usare i modelli e le variabili definiti in fourth_task.ipynb
test_pred_dt = dt.predict(x_test)
test_pred_rf = rf.predict(x_test)

### Decision Tree

#### Feature importance

In [ ]:
# Supponiamo di avere un Random Forest addestrato: dt
feature_importance = pd.Series(dt.feature_importances_, index=x_train.columns)
feature_importance = feature_importance.sort_values(ascending=False)

# Visualizzazione
feature_importance.plot(kind='bar', title="Feature Importance")

#### Rule explanation

#### Rule explanation

### Random Forest

#### Feature importance

#### Rule explanation

### Naïve Bayes

#### Feature importance

#### Rule explanation

#### Counterfactual instances

### K-Nearest Neighbors (KNN)

#### Feature importance

#### Rule explanation

#### Counterfactual instances

### Neural Network

#### Feature importance

#### Rule explanation

#### Counterfactual instances